In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import SGD
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

os.chdir('../Utils/')
import featureGenerator
from featureGenerator import *


# Generate Features and Response Vars

In [2]:
data_dir = '../../ProjectData/'
in_path = data_dir+'msft-orderbook.csv'
out_path = data_dir+'msft-orderbook-v2.csv'

createFeatures(in_path, out_path)

/Users/adam/Desktop/Stanford/MSE 448/448 Project/Utils/featureGenerator.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['Response'] = response_col


In [40]:
data = pd.read_csv(out_path)
data = data.drop(['datetime', 'direct.last_SRO'], axis = 1)
dataset = data.values
dataset = dataset.astype('float32')

# normalize the dataset
#scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)

In [41]:
data.head()

,direct.bid1,direct.ask1,direct.bsize1,direct.asize1,direct.bnum1,direct.anum1,direct.bid2,direct.ask2,direct.bsize2,direct.asize2,...,direct.anum10,direct.last_price,direct.vwap,direct.spread,direct.mid,direct.wmid,direct.volume,direct.notional,direct.last_size,Response
0,65.29,65.32,53100,17,7,1,65.27,65.35,2,100,...,3,65.29,65.250597,0.03,65.305,65.319990,17805,1161786.88,500,65.300
1,65.29,65.31,2412,505,4,2,65.27,65.32,715,821,...,2,65.29,65.283737,0.02,65.300,65.306538,98044,6400678.71,100,65.240
2,65.23,65.25,2123,80,17,1,65.22,65.26,200,200,...,2,65.25,65.281739,0.02,65.240,65.249274,107198,6998071.81,5,65.195
3,65.18,65.21,276,100,3,1,65.17,65.22,451,100,...,5,65.20,65.278454,0.03,65.195,65.202021,112623,7351855.35,100,65.170
4,65.16,65.18,272,100,3,1,65.15,65.19,1820,100,...,2,65.16,65.276766,0.02,65.170,65.174624,114829,7495665.82,28,65.170


# Train/Test Split

In [42]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

15677 7722


In [43]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [44]:
test[0]

array([  6.51900024e+01,   6.51999969e+01,   5.98000000e+03,
         5.38700000e+03,   4.70000000e+01,   4.00000000e+01,
         6.51800003e+01,   6.52099991e+01,   6.89200000e+03,
         6.76100000e+03,   5.60000000e+01,   4.90000000e+01,
         6.51699982e+01,   6.52200012e+01,   8.42400000e+03,
         6.20600000e+03,   5.80000000e+01,   4.80000000e+01,
         6.51600037e+01,   6.52300034e+01,   6.77900000e+03,
         7.57000000e+03,   5.40000000e+01,   5.70000000e+01,
         6.51500015e+01,   6.52399979e+01,   7.73700000e+03,
         6.01300000e+03,   5.80000000e+01,   5.00000000e+01,
         6.51399994e+01,   6.52500000e+01,   1.80120000e+04,
         5.85400000e+03,   5.60000000e+01,   5.00000000e+01,
         6.51299973e+01,   6.52600021e+01,   5.01300000e+03,
         5.16300000e+03,   4.20000000e+01,   5.00000000e+01,
         6.51200027e+01,   6.52699966e+01,   5.40000000e+03,
         3.89600000e+03,   3.30000000e+01,   3.30000000e+01,
         6.51100006e+01,

In [51]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

Epoch 1/100
 - 45s - loss: 2755.8597
Epoch 2/100


KeyboardInterrupt: 

In [ ]:
class OrderBookNN:
    def __init__(self, input_length, hidden)

array([['2017-04-13 09:30:00', 65.29, 65.32, ..., 500, 'INET',
        65.30000000000001],
       ['2017-04-13 09:30:01', 65.29, 65.31, ..., 100, 'INET',
        65.24000000000001],
       ['2017-04-13 09:30:02', 65.23, 65.25, ..., 5, 'BATS', 65.195],
       ..., 
       ['2017-04-13 15:59:56', 64.98, 64.99, ..., 473, 'INET', 64.975],
       ['2017-04-13 15:59:57', 64.97, 64.98, ..., 100, 'INET', 64.965],
       ['2017-04-13 15:59:58', 64.96, 64.97, ..., 146, 'EDGX', 64.965]], dtype=object)